In [146]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Sample food data in CSV (replace with actual path to your CSV file)
food_data = pd.read_csv('resisData2.csv')

# Function to calculate nutritional needs
def nutrition_need(umur, tb, bb, aktifitas, kondisi, waktu_makan):
    aktivitas_factor = {
        "Bed Rest": 1.1,
        "bergerak terbatas": 1.2,
        "bisa jalan": 1.3
    }

    if aktifitas not in aktivitas_factor:
        return False

    aktifitas = aktivitas_factor[aktifitas]

    # Calculate BMR and macronutrient needs
    BMR_perempuan = 655 + (9.6 * bb) + (1.8 * tb) - (4.7 * umur)
    total_kalori = BMR_perempuan * aktifitas
    karbohidrat = (0.6 * total_kalori) / 4
    lemak = (0.25 * total_kalori) / 9
    protein = (0.15 * total_kalori) / 4

    batas_atas = {
        'energi_kal': total_kalori,
        'karbohidrat_gram': karbohidrat,
        'lemak_gram': lemak,
        'protein_gram': protein,
        'serat_gram': 0,
        'air_gram': 0,
        'vitc_mg': 0,
        'retinol_mcg': 0,
        'kalsium_mg': 0,
        'fosfor_mg': 0,
        'zatbesi_mg': 0,
        'kalium_mg': 0,
        'natrium_mg': 0,
        'tembaga_mg': 0
    }

    age_based_limits = [
        (11, 12, {'serat_gram': 27, 'air_gram': 1850, 'vitc_mg': 50, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 1250, 'zatbesi_mg': 8, 'kalium_mg': 4400, 'natrium_mg': 1400, 'tembaga_mg': 700}),
        (13, 15, {'serat_gram': 27, 'air_gram': 2100, 'vitc_mg': 65, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 4800, 'natrium_mg': 1500, 'tembaga_mg': 795}),
        (16, 18, {'serat_gram': 29, 'air_gram': 2150, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 5000, 'natrium_mg': 1600, 'tembaga_mg': 890}),
        (19, 29, {'serat_gram': 32, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000,
                  'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
        (30, 49, {'serat_gram': 30, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000,
                  'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
        (50, 64, {'serat_gram': 25, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1400, 'tembaga_mg': 900}),
        (65, 80, {'serat_gram': 22, 'air_gram': 1550, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1200, 'tembaga_mg': 900}),
        (80, float('inf'), {'serat_gram': 20, 'air_gram': 1400, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                            'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1000, 'tembaga_mg': 900})
    ]

    for min_age, max_age, limits in age_based_limits:
        if min_age <= umur <= max_age:
            batas_atas.update(limits)
            break

    kondisi_based_adjustments = {
        'hamil_trim_1': {'energi_kal': 180, 'karbohidrat_gram': 25, 'lemak_gram': 2.3, 'protein_gram': 1, 'serat_gram': 3,
                         'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'tembaga_mg': 100},
        'hamil_trim_2': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 10, 'serat_gram': 4,
                         'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
        'hamil_trim_3': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 30, 'serat_gram': 4,
                         'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
        'menyusui_6_awal': {'energi_kal': 330, 'karbohidrat_gram': 45, 'lemak_gram': 2.2, 'protein_gram': 20, 'serat_gram': 5,
                            'air_gram': 800, 'vitc_mg': 45, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400,
                            'tembaga_mg': 100},
        'menyusui_6_kedua': {'energi_kal': 400, 'karbohidrat_gram': 55, 'lemak_gram': 2.4, 'protein_gram': 15, 'serat_gram': 5,
                             'air_gram': 700, 'vitc_mg': 25, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400,
                             'tembaga_mg': 100}
    }

    if kondisi in kondisi_based_adjustments:
        adjustments = kondisi_based_adjustments[kondisi]
        for key, value in adjustments.items():
            batas_atas[key] += value

    # Membagi batas atas dengan waktu makan
    for key in batas_atas:
        batas_atas[key] /= waktu_makan

    return batas_atas

# Function to recommend plates
def recommend_plates(nutritional_needs, food_data):
    required_columns = ['energi_kal', 'karbohidrat_gram', 'protein_gram', 'lemak_gram']
    
    X = food_data[required_columns]
    
    knn = NearestNeighbors(n_neighbors=10)
    knn.fit(X)
    
    staple_foods = food_data[food_data['jenis_pangan'] == 'serealia']
    side_dishes = food_data[food_data['jenis_pangan'].isin(['ikan kerang udang', 'daging unggas', 'kacang biji bean'])]
    fruits = food_data[food_data['jenis_pangan'] == 'buah']
    vegetables = food_data[food_data['jenis_pangan'] == 'sayuran']
    
    recommendations = []
    
    for _ in range(3):  # Generate 3 plates
        plate = []
        
        staple_food = staple_foods.sample(n=1)
        side_dish = side_dishes.sample(n=1)
        fruit = fruits.sample(n=1)
        vegetable = vegetables.sample(n=1)
        
        plate.extend(staple_food['nama_bahan'].values)
        plate.extend(side_dish['nama_bahan'].values)
        plate.extend(fruit['nama_bahan'].values)
        plate.extend(vegetable['nama_bahan'].values)
        
        recommendations.append(plate)
    
    return recommendations

# Example usage
umur = 30
tb = 160
bb = 60
aktifitas = 'bisa jalan'
kondisi = 'hamil_trim_1'
waktu_makan = 3

nutritional_needs = nutrition_need(umur, tb, bb, aktifitas, kondisi, waktu_makan)
plates = recommend_plates(nutritional_needs, food_data)

print("Recommended Plates:")
for i, plate in enumerate(plates, start=1):
    print(f"Plate {i}: {plate}")


Recommended Plates:
Plate 1: ['Perdekel Tahu Bawang', 'Tempe Kecap', 'kedondong masak segar', 'taoge segar']
Plate 2: ['japilus', 'sop daging sapi masakan', 'matoa segar', 'daun kol sawi segar']
Plate 3: ['Kue Pisang Kukus', 'koro wedus biji kering', 'purut segar', 'Capcay Toge']


In [148]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Fungsi nutrition_need() yang telah Anda berikan
def nutrition_need(umur, tb, bb, aktifitas, kondisi, waktu_makan):
    aktivitas_factor = {
        "Bed Rest": 1.1,
        "bergerak terbatas": 1.2,
        "bisa jalan": 1.3
    }

    if aktifitas not in aktivitas_factor:
        return False

    aktifitas = aktivitas_factor[aktifitas]

    # Calculate BMR and macronutrient needs
    BMR_perempuan = 655 + (9.6 * bb) + (1.8 * tb) - (4.7 * umur)
    total_kalori = BMR_perempuan * aktifitas
    karbohidrat = (0.6 * total_kalori) / 4
    lemak = (0.25 * total_kalori) / 9
    protein = (0.15 * total_kalori) / 4

    batas_atas = {
        'energi_kal': total_kalori,
        'karbohidrat_gram': karbohidrat,
        'lemak_gram': lemak,
        'protein_gram': protein,
        'serat_gram': 0,
        'air_gram': 0,
        'vitc_mg': 0,
        'retinol_mcg': 0,
        'kalsium_mg': 0,
        'fosfor_mg': 0,
        'zatbesi_mg': 0,
        'kalium_mg': 0,
        'natrium_mg': 0,
        'tembaga_mg': 0
    }

    age_based_limits = [
        (11, 12, {'serat_gram': 27, 'air_gram': 1850, 'vitc_mg': 50, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 1250, 'zatbesi_mg': 8, 'kalium_mg': 4400, 'natrium_mg': 1400, 'tembaga_mg': 700}),
        (13, 15, {'serat_gram': 27, 'air_gram': 2100, 'vitc_mg': 65, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 4800, 'natrium_mg': 1500, 'tembaga_mg': 795}),
        (16, 18, {'serat_gram': 29, 'air_gram': 2150, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 5000, 'natrium_mg': 1600, 'tembaga_mg': 890}),
        (19, 29, {'serat_gram': 32, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000,
                  'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
        (30, 49, {'serat_gram': 30, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000,
                  'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
        (50, 64, {'serat_gram': 25, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1400, 'tembaga_mg': 900}),
        (65, 80, {'serat_gram': 22, 'air_gram': 1550, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                  'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1200, 'tembaga_mg': 900}),
        (80, float('inf'), {'serat_gram': 20, 'air_gram': 1400, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
                            'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1000, 'tembaga_mg': 900})
    ]

    for min_age, max_age, limits in age_based_limits:
        if min_age <= umur <= max_age:
            batas_atas.update(limits)
            break

    kondisi_based_adjustments = {
        'hamil_trim_1': {'energi_kal': 180, 'karbohidrat_gram': 25, 'lemak_gram': 2.3, 'protein_gram': 1, 'serat_gram': 3,
                         'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'tembaga_mg': 100},
        'hamil_trim_2': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 10, 'serat_gram': 4,
                         'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
        'hamil_trim_3': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 30, 'serat_gram': 4,
                         'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
        'menyusui_6_awal': {'energi_kal': 330, 'karbohidrat_gram': 45, 'lemak_gram': 2.2, 'protein_gram': 20, 'serat_gram': 5,
                            'air_gram': 800, 'vitc_mg': 45, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400,
                            'tembaga_mg': 100},
        'menyusui_6_kedua': {'energi_kal': 400, 'karbohidrat_gram': 55, 'lemak_gram': 2.4, 'protein_gram': 15, 'serat_gram': 5,
                             'air_gram': 700, 'vitc_mg': 25, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400,
                             'tembaga_mg': 100}
    }

    if kondisi in kondisi_based_adjustments:
        adjustments = kondisi_based_adjustments[kondisi]
        for key, value in adjustments.items():
            batas_atas[key] += value

    # Membagi batas atas dengan waktu makan
    for key in batas_atas:
        batas_atas[key] /= waktu_makan

    return batas_atas

# Fungsi untuk memuat data makanan
def load_food_data(file_path):
    return pd.read_csv(file_path)

# Fungsi untuk menormalisasi data
def normalize_data(df):
    columns_to_normalize = ['energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram', 'serat_gram', 'vitc_mg', 'kalsium_mg']
    df_normalized = df[columns_to_normalize].copy()
    for column in columns_to_normalize:
        df_normalized[column] = (df_normalized[column] - df_normalized[column].min()) / (df_normalized[column].max() - df_normalized[column].min())
    return df_normalized


# Fungsi untuk menemukan makanan terdekat menggunakan KNN
def find_nearest_food(nutrition_needs, food_data, n_neighbors=5):
    columns_to_use = ['energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram', 'serat_gram', 'vitc_mg', 'kalsium_mg']
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
    knn.fit(food_data[columns_to_use])
    distances, indices = knn.kneighbors([nutrition_needs])
    return indices[0]


# Fungsi untuk membuat rekomendasi piring
def create_plate_recommendation(nutrition_needs, food_data):
    plate = []
    
    # Makanan pokok (serealia)
    carbs_need = (nutrition_needs['karbohidrat_gram'] * 4) * (2/6)
    serealia = food_data[food_data['jenis_pangan'] == 'serealia']
    serealia_indices = find_nearest_food([carbs_need, 0, 0, carbs_need, 0, 0, 0], serealia)
    plate.append(serealia.iloc[serealia_indices[0]]['nama_bahan'])
    
    # Lauk pauk
    protein_need = (nutrition_needs['protein_gram'] * 4) * (1/6)
    lauk = food_data[food_data['jenis_pangan'].isin(['ikan kerang udang', 'daging unggas', 'kacang biji bean'])]
    lauk_indices = find_nearest_food([protein_need * 4, protein_need, 0, 0, 0, 0, 0], lauk)
    plate.append(lauk.iloc[lauk_indices[0]]['nama_bahan'])
    
    # Buah
    fruit_need = nutrition_needs['energi_kal'] * (1/6)
    buah = food_data[food_data['jenis_pangan'] == 'buah']
    buah_indices = find_nearest_food([fruit_need, 0, 0, 0, 0, nutrition_needs['vitc_mg'], 0], buah)
    plate.append(buah.iloc[buah_indices[0]]['nama_bahan'])
    
    # Sayuran
    veggie_need = nutrition_needs['energi_kal'] * (2/6)
    sayur = food_data[food_data['jenis_pangan'] == 'sayuran']
    sayur_indices = find_nearest_food([veggie_need, 0, 0, 0, nutrition_needs['serat_gram'], nutrition_needs['vitc_mg'], nutrition_needs['kalsium_mg']], sayur)
    plate.append(sayur.iloc[sayur_indices[0]]['nama_bahan'])
    
    return plate

# Fungsi utama untuk menghasilkan rekomendasi
def generate_recommendations(umur, tb, bb, aktifitas, kondisi, waktu_makan, food_data_path, num_recommendations=3):
    # Hitung kebutuhan nutrisi
    nutrition_needs = nutrition_need(umur, tb, bb, aktifitas, kondisi, waktu_makan)
    
    # Muat data makanan
    food_data = load_food_data(food_data_path)
    
    # Normalisasi data makanan
    normalized_food_data = normalize_data(food_data)
    
    # Gabungkan data ternormalisasi dengan data asli
    food_data_normalized = pd.concat([food_data, normalized_food_data], axis=1)
    
    # Buat rekomendasi
    recommendations = []
    for _ in range(num_recommendations):
        plate = create_plate_recommendation(nutrition_needs, food_data_normalized)
        recommendations.append(plate)
    
    return recommendations

# Contoh penggunaan
umur = 30
tb = 165
bb = 60
aktifitas = "bisa jalan"
kondisi = "normal"
waktu_makan = 3
food_data_path = "resisData2.csv"

recommendations = generate_recommendations(umur, tb, bb, aktifitas, kondisi, waktu_makan, food_data_path)

print("Rekomendasi Piring:")
for i, plate in enumerate(recommendations, 1):
    print(f"Piring {i}: {plate}")

/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


ValueError: X has 7 features, but NearestNeighbors is expecting 14 features as input.

In [134]:
import pandas as pd

# Load data
data = pd.read_csv('resisData.csv')

# Function to recommend food for a plate
def rekomendasi_makanan(data):
    max_calories = 700
    batas_pokok = (2/6) * max_calories
    batas_lauk = (1/6) * max_calories
    batas_buah = (1/6) * max_calories
    batas_sayur = (2/6) * max_calories
    
    while True:
        # Filtering data based on food categories
        makanan_pokok = data[data['jenis_pangan'].isin(['serealia', 'umbi berpati'])]
        lauk_pauk = data[data['jenis_pangan'].isin(['ikan kerang udang', 'daging unggas', 'kacang biji bean'])]
        buah = data[data['jenis_pangan'] == 'buah']
        sayuran = data[data['jenis_pangan'] == 'sayuran']

        # Randomly select one item from each category
        makanan_pokok_selected = makanan_pokok.sample(n=1)
        lauk_pauk_selected = lauk_pauk.sample(n=1)
        buah_selected = buah.sample(n=1)
        sayuran_selected = sayuran.sample(n=1)

        # Calculate calories for each category
        kalori_pokok = makanan_pokok_selected.iloc[0]['energi_kal']
        kalori_lauk = lauk_pauk_selected.iloc[0]['energi_kal']
        kalori_buah = buah_selected.iloc[0]['energi_kal']
        kalori_sayur = sayuran_selected.iloc[0]['energi_kal']

        # Calculate total calories
        total_calories = kalori_pokok + kalori_lauk + kalori_buah + kalori_sayur
        
        # Check if the calories for each category are within the allowed limit
        if (kalori_pokok <= batas_pokok and
            kalori_lauk <= batas_lauk and
            kalori_buah <= batas_buah and
            kalori_sayur <= batas_sayur and
            total_calories <= max_calories):
            plate = {
                'makanan pokok': makanan_pokok_selected.iloc[0]['nama_bahan'],
                'lauk pauk': lauk_pauk_selected.iloc[0]['nama_bahan'],
                'buah': buah_selected.iloc[0]['nama_bahan'],
                'sayur-sayuran': sayuran_selected.iloc[0]['nama_bahan'],
                'total kalori': total_calories
            }
            return plate

# Generate recommendations
rekomendasi_piring = []
for i in range(3):  # Generate 3 plates
    rekomendasi_piring.append(rekomendasi_makanan(data))

# Output recommendations
for idx, piring in enumerate(rekomendasi_piring, 1):
    print(f'Piring {idx}')
    for kategori, makanan in piring.items():
        print(f'{kategori} => {makanan}')

Piring 1
makanan pokok => Promina milky beras merah
lauk pauk => sapi babat segar
buah => biwah segar
sayur-sayuran => lumai leunca segar
total kalori => 407.0
Piring 2
makanan pokok => bubur tinotuan (manado)
lauk pauk => soto jeroan masakan
buah => mangga kopek segar
sayur-sayuran => terung panjang
total kalori => 329.0
Piring 3
makanan pokok => batatas tali ubi rebus
lauk pauk => sapi dideh darah
buah => nanas segar
sayur-sayuran => taoge seduh
total kalori => 348.0


In [81]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

# Parameters
algo = "auto"
met = "cosine"

# Load data
data = pd.read_csv('resisData2.csv')

# Drop unnecessary columns
data = data.drop(['id', 'kode', 'sumber', 'gambar', 'satuan'], axis=1)
data['jenis_pangan_encoded'] = pd.factorize(data['jenis_pangan'])[0]

# Define numeric columns
numeric_cols = ["jenis_pangan_encoded", "air_gram", "energi_kal", "protein_gram", "lemak_gram", "karbohidrat_gram",
                "serat_gram", "natrium_mg", "kalium_mg", "bdd"]

# Replace commas with dots in numeric columns and convert to float
data[numeric_cols] = data[numeric_cols].replace({',': '.'}, regex=True)
data[numeric_cols] = data[numeric_cols].astype(float)

# Normalize data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data[numeric_cols])

# Using Nearest Neighbors
model = NearestNeighbors(n_neighbors=6, algorithm=algo, metric=met)
model.fit(data_normalized)

# Function to get food recommendations
def get_recommendations(food_names, allergy_list):
    # Find indices of the given food names
    food_indices = [data[data['nama_bahan'] == food_name].index[0] for food_name in food_names]
    
    # Find nearest neighbors
    distances, indices = model.kneighbors(data_normalized[food_indices])
    
    # Get food names and distances
    recommendations = []
    jarak = []
    for idx_list, dist_list in zip(indices, distances):
        for idx, dist in zip(idx_list, dist_list):
            if idx not in food_indices and data.iloc[idx]['nama_bahan'] not in allergy_list:
                recommendations.append(data.iloc[idx]['nama_bahan'])
                jarak.append(dist)
    
    return recommendations[:10], jarak[:10]

# Function to recommend food for a plate
def rekomendasi_piring(user_foods, n_recommendations=3):
    rekomendasi = []
    for food in user_foods:
        recs, _ = get_recommendations([food], [])
        rekomendasi.append(recs[:n_recommendations])
    
    plates = []
    for i in range(n_recommendations):
        plate = []
        for rec_list in rekomendasi:
            plate.append(rec_list[i])
        plates.append(plate)
    
    return plates

# Contoh penggunaan
user_foods = ['Nasi Goreng', 'apel segar', 'Ikan salmon', 'bayam rebus']
recommendations = rekomendasi_piring(user_foods)

# Output recommendations
for idx, plate in enumerate(recommendations, 1):
    print(f'Piring {idx}')
    for food in plate:
        print(f' - {food}')


Piring 1
 - Roti Pisang Oat
 - jambu air segar
 - Ikan tuna
 - caisin segar
Piring 2
 - Mr. Bread Roti Sandwich krim keju
 - jambu monyet segar
 - ikan mujahir pepes
 - tomat air (sari) segar
Piring 3
 - Mr. Bread Roti manis gandum keju
 - apel malang segar
 - takwa
 - paria putih kukus


In [87]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Load data makanan dan nutrisi dari CSV
df = pd.read_csv('resisData2.csv')

# Contoh data input user
user_data = {
    'age': 30,
    'height': 160,
    'weight': 60,
    'activity': 'moderate',
    'condition': 'pregnant',
    'meals_per_day': 3,
    'allergies': ['daging unggas']
}

# Fungsi untuk menghitung batasan kalori per kategori makanan
def calculate_food_limits(max_calories):
    batas_pokok = (2/6) * max_calories
    batas_lauk = (1/6) * max_calories
    batas_buah = (1/6) * max_calories
    batas_sayur = (2/6) * max_calories
    return batas_pokok, batas_lauk, batas_buah, batas_sayur

# Fungsi untuk mendapatkan rekomendasi piring makanan
def get_food_recommendations(user_data, df):
    max_calories = 700
    batas_pokok, batas_lauk, batas_buah, batas_sayur = calculate_food_limits(max_calories)
    
    # Filter makanan berdasarkan alergi
    filtered_df = df[~df['jenis_pangan'].isin(user_data['allergies'])]
    
    # KNN untuk rekomendasi makanan
    knn = NearestNeighbors(n_neighbors=3, algorithm='auto').fit(filtered_df[['energi_kal']])
    
    # Membangun piring makanan
    plates = []
    for i in range(user_data['meals_per_day']):
        # Sample makanan untuk setiap kategori
        try:
            staples = filtered_df[(filtered_df['jenis_pangan'] == 'serealia') & (filtered_df['energi_kal'] <= batas_pokok)].sample(1)
        except ValueError:
            staples = df[(df['jenis_pangan'] == 'serealia') & (df['energi_kal'] <= batas_pokok)].sample(1)
        
        try:
            sides = filtered_df[(filtered_df['jenis_pangan'] == 'daging unggas') & (filtered_df['energi_kal'] <= batas_lauk)].sample(1)
        except ValueError:
            sides = df[(df['jenis_pangan'] == 'daging unggas') & (df['energi_kal'] <= batas_lauk)].sample(1)
        
        try:
            fruits = filtered_df[(filtered_df['jenis_pangan'] == 'buah') & (filtered_df['energi_kal'] <= batas_buah)].sample(1)
        except ValueError:
            fruits = df[(df['jenis_pangan'] == 'buah') & (df['energi_kal'] <= batas_buah)].sample(1)
        
        try:
            veggies = filtered_df[(filtered_df['jenis_pangan'] == 'sayuran') & (filtered_df['energi_kal'] <= batas_sayur)].sample(1)
        except ValueError:
            veggies = df[(df['jenis_pangan'] == 'sayuran') & (df['energi_kal'] <= batas_sayur)].sample(1)
        
        plate = pd.concat([staples, sides, fruits, veggies])
        plates.append(plate)
    
    return plates

# Mendapatkan rekomendasi makanan
recommended_plates = get_food_recommendations(user_data, df)

# Output rekomendasi piring makanan
for i, plate in enumerate(recommended_plates, 1):
    plate_foods = plate['nama_bahan'].tolist()
    total_calories = plate['energi_kal'].sum()
    print(f"Plate {i}: {', '.join(plate_foods)} | Total Calories: {total_calories:.2f}")


Plate 1: Roma sari gandum, soto sulung masakan, buah mentega (bisbul) segar, daun tespong segar | Total Calories: 295.00
Plate 2: Pancake Oat, Omelet Tuna, mangga muda segar, daun lompong talas segar | Total Calories: 390.00
Plate 3: kapusa, sapi babat segar, vigus segar, brokoli , matang | Total Calories: 333.20


In [2]:
import random
import pandas as pd

# Definisikan makanan dan nilai gizi
foods = pd.read_csv('resisData2.csv')

# Definisikan parameter genetik
population_size = 10
num_generations = 50
mutation_rate = 0.1

# Buat populasi awal
def create_individual():
    return random.sample(list(foods.keys()), 4)

def create_population(size):
    return [create_individual() for _ in range(size)]

# Hitung kebugaran individu berdasarkan kesesuaian nilai gizi
def fitness(individual, target_nutrition):
    total_nutrition = {'energi_kal': 0, 'protein': 0, 'lemak': 0, 'karbohidrat': 0}
    for food in individual:
        for key in total_nutrition:
            total_nutrition[key] += foods[food][key]
    
    fitness_value = 0
    for key in total_nutrition:
        fitness_value += abs(total_nutrition[key] - target_nutrition[key])
    
    return fitness_value

# Seleksi individu berdasarkan kebugaran
def selection(population, target_nutrition):
    sorted_population = sorted(population, key=lambda x: fitness(x, target_nutrition))
    return sorted_population[:2]

# Crossover untuk menghasilkan keturunan
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return [child1, child2]

# Mutasi individu
def mutate(individual):
    if random.random() < mutation_rate:
        index = random.randint(0, len(individual) - 1)
        new_food = random.choice(list(foods.keys()))
        individual[index] = new_food
    return individual

# Algoritma genetika utama
def genetic_algorithm(target_nutrition):
    population = create_population(population_size)
    
    for generation in range(num_generations):
        new_population = []
        selected_individuals = selection(population, target_nutrition)
        
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(selected_individuals, 2)
            children = crossover(parent1, parent2)
            new_population.extend(children)
        
        population = [mutate(individual) for individual in new_population]
    
    best_individual = selection(population, target_nutrition)[0]
    return best_individual

# Contoh input target nilai gizi
target_nutrition = {'energi_kal': 700, 'protein': 50, 'lemak': 20, 'karbohidrat': 100}

# Jalankan algoritma genetika untuk mendapatkan rekomendasi makanan
recommended_meal = genetic_algorithm(target_nutrition)
print("Rekomendasi makanan:", recommended_meal)


KeyError: 'energi_kal'

In [151]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Load data makanan dan nutrisi dari CSV
df = pd.read_csv('resisData2.csv')

# Contoh data input user
user_data = {
    'age': 30,
    'height': 160,
    'weight': 60,
    'activity': 'moderate',
    'condition': 'pregnant',
    'meals_per_day': 3,
    'allergies': ['daging unggas']
}

# Fungsi untuk menghitung batasan kalori per kategori makanan
def calculate_food_limits(max_calories):
    batas_pokok = (2/6) * max_calories
    batas_lauk = (1/6) * max_calories
    batas_buah = (1/6) * max_calories
    batas_sayur = (2/6) * max_calories
    return batas_pokok, batas_lauk, batas_buah, batas_sayur

# Fungsi untuk mendapatkan rekomendasi piring makanan
def get_food_recommendations(user_data, df):
    max_calories = 700
    batas_pokok, batas_lauk, batas_buah, batas_sayur = calculate_food_limits(max_calories)
    
    # Filter makanan berdasarkan alergi
    filtered_df = df[~df['jenis_pangan'].isin(user_data['allergies'])]
    
    # Membangun piring makanan
    plates = []
    for i in range(user_data['meals_per_day']):
        # Sample makanan untuk setiap kategori menggunakan KNN
        try:
            knn_pokok = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(filtered_df[filtered_df['jenis_pangan'] == 'serealia'][['energi_kal']])
            distances, indices = knn_pokok.kneighbors([[batas_pokok]])
            staples = filtered_df.iloc[indices[0]]
        except ValueError:
            staples = df[(df['jenis_pangan'] == 'serealia') & (df['energi_kal'] <= batas_pokok)].sample(1)
        
        try:
            knn_lauk = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(filtered_df[filtered_df['jenis_pangan'] == 'daging unggas'][['energi_kal']])
            distances, indices = knn_lauk.kneighbors([[batas_lauk]])
            sides = filtered_df.iloc[indices[0]]
        except ValueError:
            sides = df[(df['jenis_pangan'] == 'daging unggas') & (df['energi_kal'] <= batas_lauk)].sample(1)
        
        try:
            knn_buah = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(filtered_df[filtered_df['jenis_pangan'] == 'buah'][['energi_kal']])
            distances, indices = knn_buah.kneighbors([[batas_buah]])
            fruits = filtered_df.iloc[indices[0]]
        except ValueError:
            fruits = df[(df['jenis_pangan'] == 'buah') & (df['energi_kal'] <= batas_buah)].sample(1)
        
        try:
            knn_sayur = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(filtered_df[filtered_df['jenis_pangan'] == 'sayuran'][['energi_kal']])
            distances, indices = knn_sayur.kneighbors([[batas_sayur]])
            veggies = filtered_df.iloc[indices[0]]
        except ValueError:
            veggies = df[(df['jenis_pangan'] == 'sayuran') & (df['energi_kal'] <= batas_sayur)].sample(1)
        
        plate = pd.concat([staples, sides, fruits, veggies])
        plates.append(plate)
    
    return plates

# Mendapatkan rekomendasi makanan
recommended_plates = get_food_recommendations(user_data, df)

# Output rekomendasi piring makanan
for i, plate in enumerate(recommended_plates, 1):
    plate_foods = plate['nama_bahan'].tolist()
    total_calories = plate['energi_kal'].sum()
    print(f"Plate {i}: {', '.join(plate_foods)} | Total Calories: {total_calories:.2f}")


Plate 1: Sup Pembakar Lemak, soto banjar masakan, Enoki Pedas Manis, Puding Chia | Total Calories: 335.00
Plate 2: Sup Pembakar Lemak, Ayam brand tuna dalam air, Enoki Pedas Manis, Puding Chia | Total Calories: 275.00
Plate 3: Sup Pembakar Lemak, Dada ayam goreng dengan kulit, Enoki Pedas Manis, Puding Chia | Total Calories: 333.00


/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [118]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import random

# Baca data makanan dari CSV
data_makanan = pd.read_csv('resisData3.csv')

# Pisahkan data menjadi kategori 'Isi Piringku'
makanan_pokok = data_makanan[data_makanan['jenis_pangan'] == 'serealia']
lauk_pauk = data_makanan[(data_makanan['jenis_pangan'] == 'ikan kerang udang') | 
                         (data_makanan['jenis_pangan'] == 'daging unggas') | 
                         (data_makanan['jenis_pangan'] == 'kacang biji bean')]
buah = data_makanan[data_makanan['jenis_pangan'] == 'buah']
sayuran = data_makanan[data_makanan['jenis_pangan'] == 'sayuran']

# Fungsi untuk rekomendasi makanan
def rekomendasi_makanan(input_plate, data_kategori, fitur, n_neighbors=5):
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
    X = data_kategori[fitur]
    knn.fit(X)
    input_values = pd.DataFrame([input_plate], columns=fitur)
    distances, indices = knn.kneighbors(input_values)
    rekomendasi = data_kategori.iloc[indices[0]]
    return rekomendasi.sample(n=1)  # Pilih satu secara acak dari n_neighbors terdekat

def generate_piring_rekomendasi(target_kalori, toleransi_bawah=50):
    while True:
        # Acak proporsi kalori untuk setiap kategori
        proporsi = [random.uniform(0.3, 0.4), random.uniform(0.1, 0.2), 
                    random.uniform(0.1, 0.2), random.uniform(0.2, 0.4)]
        proporsi = [p / sum(proporsi) for p in proporsi]  # Normalisasi

        input_plate_pokok = {
            'karbohidrat_gram': (0.6 * target_kalori * proporsi[0]) / 4
        }
        input_plate_lauk = {
            'protein_gram': (0.15 * target_kalori * proporsi[1]) / 4
        }
        input_plate_buah = {
            'energi_kal': target_kalori * proporsi[2]
        }
        input_plate_sayur = {
            'energi_kal': target_kalori * proporsi[3]
        }
        
        

        rekom_pokok = rekomendasi_makanan(input_plate_pokok, makanan_pokok, ['karbohidrat_gram'])
        rekom_lauk = rekomendasi_makanan(input_plate_lauk, lauk_pauk, ['protein_gram'])
        rekom_buah = rekomendasi_makanan(input_plate_buah, buah, ['energi_kal'])
        rekom_sayuran = rekomendasi_makanan(input_plate_sayur, sayuran, ['energi_kal'])

        total_kalori = (rekom_pokok['energi_kal'].values[0] +
                        rekom_lauk['energi_kal'].values[0] +
                        rekom_buah['energi_kal'].values[0] +
                        rekom_sayuran['energi_kal'].values[0])

        if target_kalori - toleransi_bawah <= total_kalori <= target_kalori:
            return {
                'makanan_pokok': rekom_pokok['nama_bahan'].values[0],
                'lauk_pauk': rekom_lauk['nama_bahan'].values[0],
                'buah': rekom_buah['nama_bahan'].values[0],
                'sayuran': rekom_sayuran['nama_bahan'].values[0],
                'total_kalori': total_kalori
            }

# Fungsi untuk simulasi klik submit
def klik_submit(target_kalori=700):
    return generate_piring_rekomendasi(target_kalori)

# Contoh penggunaan
for i in range(3):  # Simulasi 5 kali klik submit
    print(f"Rekomendasi ke-{i+1}:")
    print(klik_submit())
    print()

Rekomendasi ke-1:
{'makanan_pokok': 'Pancake Pisang', 'lauk_pauk': 'gulai kambing', 'buah': 'pisang kayu segar', 'sayuran': 'Tumis Sawi Putih', 'total_kalori': 684.0}

Rekomendasi ke-2:
{'makanan_pokok': 'Mr. Bread Roti tawar choco chips', 'lauk_pauk': 'soto pekalongan masakan', 'buah': 'sukun tua segar', 'sayuran': 'Mangkuk Smoothie Hijau', 'total_kalori': 692.0}

Rekomendasi ke-3:
{'makanan_pokok': 'bakwan', 'lauk_pauk': 'ampas tahu kering', 'buah': 'mangga manalagi segar', 'sayuran': 'Ayam gulai', 'total_kalori': 690.0}



In [3]:
import random
import pandas as pd
import numpy as np
from typing import List, Dict, Union
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

def nutrition_need(umur: int, tb: float, bb: float, aktifitas: str, kondisi: str, waktu_makan: int) -> Dict[str, float]:
    aktivitas_factor = {
        "Bed Rest": 1.1,
        "bergerak terbatas": 1.2,
        "bisa jalan": 1.3
    }

    if aktifitas not in aktivitas_factor:
        return {}

    aktifitas = aktivitas_factor[aktifitas]

    # Calculate BMR and macronutrient needs
    BMR_perempuan = 655 + (9.6 * bb) + (1.8 * tb) - (4.7 * umur)
    total_kalori = BMR_perempuan * aktifitas
    karbohidrat = (0.6 * total_kalori) / 4
    lemak = (0.25 * total_kalori) / 9
    protein = (0.15 * total_kalori) / 4

    batas_atas = {
        'energi_kal': total_kalori,
        'karbohidrat_gram': karbohidrat,
        'lemak_gram': lemak,
        'protein_gram': protein,
        'serat_gram': 0,
        'air_gram': 0,
        'vitc_mg': 0,
        'retinol_mcg': 0,
        'kalsium_mg': 0,
        'fosfor_mg': 0,
        'zatbesi_mg': 0,
        'kalium_mg': 0,
        'natrium_mg': 0,
        'tembaga_mg': 0
    }

    age_based_limits = [
        (11, 12, {'serat_gram': 27, 'air_gram': 1850, 'vitc_mg': 50, 'retinol_mcg': 600, 'kalsium_mg': 1200,
         'fosfor_mg': 1250, 'zatbesi_mg': 8, 'kalium_mg': 4400, 'natrium_mg': 1400, 'tembaga_mg': 700}),
        (13, 15, {'serat_gram': 27, 'air_gram': 2100, 'vitc_mg': 65, 'retinol_mcg': 600, 'kalsium_mg': 1200,
         'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 4800, 'natrium_mg': 1500, 'tembaga_mg': 795}),
        (16, 18, {'serat_gram': 29, 'air_gram': 2150, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
         'fosfor_mg': 1250, 'zatbesi_mg': 15, 'kalium_mg': 5000, 'natrium_mg': 1600, 'tembaga_mg': 890}),
        (19, 29, {'serat_gram': 32, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000,
         'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
        (30, 49, {'serat_gram': 30, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1000,
         'fosfor_mg': 700, 'zatbesi_mg': 18, 'kalium_mg': 4700, 'natrium_mg': 1500, 'tembaga_mg': 900}),
        (50, 64, {'serat_gram': 25, 'air_gram': 2350, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
         'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1400, 'tembaga_mg': 900}),
        (65, 80, {'serat_gram': 22, 'air_gram': 1550, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
         'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1200, 'tembaga_mg': 900}),
        (80, float('inf'), {'serat_gram': 20, 'air_gram': 1400, 'vitc_mg': 75, 'retinol_mcg': 600, 'kalsium_mg': 1200,
         'fosfor_mg': 700, 'zatbesi_mg': 8, 'kalium_mg': 4700, 'natrium_mg': 1000, 'tembaga_mg': 900})
    ]

    for min_age, max_age, limits in age_based_limits:
        if min_age <= umur <= max_age:
            batas_atas.update(limits)
            break

    kondisi_based_adjustments = {
        'hamil_trim_1': {'energi_kal': 180, 'karbohidrat_gram': 25, 'lemak_gram': 2.3, 'protein_gram': 1, 'serat_gram': 3, 'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'tembaga_mg': 100},
        'hamil_trim_2': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 10, 'serat_gram': 4, 'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
        'hamil_trim_3': {'energi_kal': 300, 'karbohidrat_gram': 40, 'lemak_gram': 2.3, 'protein_gram': 30, 'serat_gram': 4, 'air_gram': 300, 'vitc_mg': 10, 'retinol_mcg': 300, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'tembaga_mg': 100},
        'menyusui_6_awal': {'energi_kal': 330, 'karbohidrat_gram': 45, 'lemak_gram': 2.2, 'protein_gram': 20, 'serat_gram': 5, 'air_gram': 800, 'vitc_mg': 45, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400, 'tembaga_mg': 100},
        'menyusui_6_kedua': {'energi_kal': 400, 'karbohidrat_gram': 55, 'lemak_gram': 2.4, 'protein_gram': 15, 'serat_gram': 5, 'air_gram': 700, 'vitc_mg': 25, 'retinol_mcg': 350, 'kalsium_mg': 200, 'zatbesi_mg': 9, 'kalium_mg': 400, 'tembaga_mg': 100}
    }

    if kondisi in kondisi_based_adjustments:
        adjustments = kondisi_based_adjustments[kondisi]
        for key, value in adjustments.items():
            batas_atas[key] += value

    # Membagi batas atas dengan waktu makan
    for key in batas_atas:
        batas_atas[key] /= waktu_makan

    return batas_atas

def load_food_data(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

def preprocess_data(data):
    # Pilih fitur nutrisi yang relevan
    nutrisi_features = ['energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram', 'serat_gram', 'vitc_mg', 'kalsium_mg']
    
    # Normalisasi data
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(data[nutrisi_features])
    
    return normalized_data, scaler

def build_knn_model(normalized_data, n_neighbors=10):
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
    knn.fit(normalized_data)
    return knn

def pilih_makanan_knn(data, normalized_data, knn_model, scaler, jenis_pangan, target_kalori, **target_nutrisi):
    if isinstance(jenis_pangan, str):
        kandidat = data[data['jenis_pangan'] == jenis_pangan]
    else:
        kandidat = data[data['jenis_pangan'].isin(jenis_pangan)]
    
    if kandidat.empty:
        return {}

    # Buat vektor target berdasarkan target nutrisi
    target_vector = [target_kalori] + [target_nutrisi.get(nutrisi, 0) for nutrisi in ['protein_gram', 'lemak_gram', 'karbohidrat_gram', 'serat_gram', 'vitc_mg', 'kalsium_mg']]
    target_normalized = scaler.transform([target_vector])

    # Temukan k tetangga terdekat
    distances, indices = knn_model.kneighbors(target_normalized)

    # Pilih secara acak dari k tetangga terdekat
    chosen_index = random.choice(indices[0])
    return data.iloc[chosen_index].to_dict()

def rekomendasi_makanan_knn(data, normalized_data, knn_model, scaler, nutrition_need, jumlah_piring):
    rekomendasi = []
    makanan_terpilih = set()  # Untuk memastikan variasi antar piring
    
    for _ in range(jumlah_piring):
        piring = []
        piring_id = set()  # Untuk memastikan variasi dalam satu piring
        
        # Pilih makanan pokok (serealia)
        while True:
            makanan_pokok = pilih_makanan_knn(data, normalized_data, knn_model, scaler, "serealia", 
                                              target_kalori=nutrition_need['energi_kal'] * 2/6,
                                              karbohidrat_gram=nutrition_need['karbohidrat_gram'] * 2/6)
            if makanan_pokok['id'] not in makanan_terpilih:
                piring.append(makanan_pokok)
                makanan_terpilih.add(makanan_pokok['id'])
                piring_id.add(makanan_pokok['id'])
                break
        
        # Pilih lauk pauk
        while True:
            lauk_pauk = pilih_makanan_knn(data, normalized_data, knn_model, scaler, 
                                          ["ikan kerang udang", "daging unggas", "kacang biji bean"],
                                          target_kalori=nutrition_need['energi_kal'] * 1/6,
                                          protein_gram=nutrition_need['protein_gram'] * 1/6)
            if lauk_pauk['id'] not in piring_id:
                piring.append(lauk_pauk)
                piring_id.add(lauk_pauk['id'])
                break
        
        # Pilih buah
        while True:
            buah = pilih_makanan_knn(data, normalized_data, knn_model, scaler, "buah",
                                     target_kalori=nutrition_need['energi_kal'] * 1/6,
                                     vitc_mg=nutrition_need['vitc_mg'] * 1/6)
            if buah['id'] not in piring_id:
                piring.append(buah)
                piring_id.add(buah['id'])
                break
        
        # Pilih sayuran
        while True:
            sayuran = pilih_makanan_knn(data, normalized_data, knn_model, scaler, "sayuran",
                                        target_kalori=nutrition_need['energi_kal'] * 2/6,
                                        vitc_mg=nutrition_need['vitc_mg'] * 2/6,
                                        kalsium_mg=nutrition_need['kalsium_mg'] * 2/6)
            if sayuran['id'] not in piring_id:
                piring.append(sayuran)
                piring_id.add(sayuran['id'])
                break
        
        rekomendasi.append(piring)
    
    return rekomendasi

def tampilkan_rekomendasi(rekomendasi: List[List[Dict]]):
    for i, piring in enumerate(rekomendasi, 1):
        print(f"Piring {i}:")
        for makanan in piring:
            if makanan:
                print(f"- {makanan['nama_bahan']} ({makanan['jenis_pangan']})")
        print()

def main():
    # Load data
    data = load_food_data('resisData3.csv')

    # Preprocess data
    normalized_data, scaler = preprocess_data(data)

    # Build KNN model
    knn_model = build_knn_model(normalized_data)

    # Input user
    umur = 25
    tb = 165
    bb = 65
    aktifitas = "bisa jalan"
    kondisi = "hamil_trim_1"
    waktu_makan = 3
    jumlah_piring = 3

    # Hitung kebutuhan nutrisi
    kebutuhan_nutrisi = nutrition_need(umur, tb, bb, aktifitas, kondisi, waktu_makan)

    # Generate rekomendasi
    rekomendasi = rekomendasi_makanan_knn(data, normalized_data, knn_model, scaler, kebutuhan_nutrisi, jumlah_piring)

    # Tampilkan rekomendasi
    tampilkan_rekomendasi(rekomendasi)

if __name__ == "__main__":
    main()

Piring 1:
- bolu peca (serealia)
- Perkedel Kentang (umbi berpati)
- Susu murni (susu)
- Susu bebelove 6-12bln (susu)

Piring 2:
- sunduk lawang (serealia)
- SUP KRIM AYAM ROYCO (serealia)
- kue nastar (serealia)
- soto betawi masakan (daging unggas)

Piring 3:
- Sohun superindo (mentah) (serealia)
- sop konro masakan (daging unggas)
- sawo duren segar (buah)
- sayur kohukohu (sayuran)



/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/fadliizurohman/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/fad

In [11]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Load and prepare food data
data = {
    'Food': ['Nasi', 'Roti', 'Sereal', 'Apel', 'Pisang', 'Ayam', 'Ikan', 'Kacang', 'Sayur Bayam', 'Sayur Wortel'],
    'Calories': [200, 150, 250, 50, 100, 300, 250, 200, 30, 40],
    'Carbohydrates': [45, 30, 55, 14, 27, 0, 0, 15, 5, 8],
    'Protein': [4, 5, 6, 0, 1, 25, 22, 8, 2, 1],
    'Fat': [0.5, 2, 1, 0, 0.3, 10, 12, 18, 0, 0],
    'Type': ['Staple', 'Staple', 'Staple', 'Fruit', 'Fruit', 'Protein', 'Protein', 'Protein', 'Vegetable', 'Vegetable']
}
food_df = pd.DataFrame(data)
food_df.to_csv('food_data.csv', index=False)

# Harris-Benedict equation to calculate BEE and TEE
def calculate_nutritional_needs(weight, height, age, activity_level, trimester):
    BEE = 655 + (9.6 * weight) + (1.8 * height) - (4.7 * age)
    PAL_dict = {'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725, 'very active': 1.9}
    PAL = PAL_dict.get(activity_level, 1.2)
    trimester_multiplier = {'first': 1.1, 'second': 1.2, 'third': 1.3}
    T_multiplier = trimester_multiplier.get(trimester, 1.1)
    TEE = BEE * PAL * T_multiplier + 300  # Adding 300 kcal for pregnancy
    carbs = 0.65 * TEE / 4
    protein = 0.15 * TEE / 4
    fat = 0.20 * TEE / 9
    return {'Calories': TEE, 'Carbohydrates': carbs, 'Protein': protein, 'Fat': fat}

# KNN algorithm for meal recommendations
food_df = pd.read_csv('food_data.csv')
X = food_df[['Calories', 'Carbohydrates', 'Protein', 'Fat']].values
knn = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(X)

def recommend_meals(nutritional_needs):
    target = np.array([[nutritional_needs['Calories'], nutritional_needs['Carbohydrates'], 
                        nutritional_needs['Protein'], nutritional_needs['Fat']]])
    distances, indices = knn.kneighbors(target)
    recommendations = food_df.iloc[indices[0]]
    return recommendations

# Full workflow example
nutritional_needs = calculate_nutritional_needs(weight=70, height=165, age=30, activity_level='light', trimester='second')
recommendations = recommend_meals(nutritional_needs)
print(recommendations)


     Food  Calories  Carbohydrates  Protein   Fat     Type
5    Ayam       300              0       25  10.0  Protein
2  Sereal       250             55        6   1.0   Staple
6    Ikan       250              0       22  12.0  Protein
0    Nasi       200             45        4   0.5   Staple
7  Kacang       200             15        8  18.0  Protein


In [37]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import random
import pandas as pd

# Data dummy makanan
# Format: [nama, kalori, protein, karbohidrat, lemak]
food_data = np.array([
    ["Nasi Putih", 200, 4, 40, 0],
    ["Telur Rebus", 70, 6, 0, 5],
    ["Apel", 95, 0, 25, 0],
    ["Susu", 150, 8, 12, 8],
    ["Ayam Panggang", 165, 31, 0, 3],
    ["Sayur Bayam", 23, 3, 4, 0],
    ["Roti Gandum", 69, 3, 12, 1],
    ["Pisang", 105, 1, 27, 0],
    ["Kacang Tanah", 160, 7, 5, 14],
    ["Ikan Salmon", 206, 22, 0, 13],
    ["Wortel", 52, 1, 12, 0],
    ["Yogurt", 150, 5, 17, 8],
    ["Kentang Rebus", 87, 2, 20, 0],
    ["Tahu", 76, 8, 2, 5],
    ["Jeruk", 62, 1, 15, 0]
])

# Fungsi untuk menghitung kebutuhan gizi ibu hamil (contoh sederhana)
def calculate_nutrient_needs(age, weight, height, trimester, activity_level):
    # Rumus Harris-Benedict untuk BMR
    bmr = 655 + (9.6 * weight) + (1.8 * height) - (4.7 * age)
    
    # Faktor aktivitas
    activity_factors = {
        "sedentary": 1.2,
        "light": 1.375,
        "moderate": 1.55,
        "active": 1.725
    }
    
    # Tambahan kalori berdasarkan trimester
    trimester_additions = {1: 0, 2: 340, 3: 450}
    
    total_calories = bmr * activity_factors[activity_level] + trimester_additions[trimester]
    
    # Distribusi makronutrien (contoh: 50% karbo, 20% protein, 30% lemak)
    carbs = (total_calories * 0.5) / 4  # 4 kalori per gram karbohidrat
    protein = (total_calories * 0.2) / 4  # 4 kalori per gram protein
    fat = (total_calories * 0.3) / 9  # 9 kalori per gram lemak
    
    return [total_calories, protein, carbs, fat]

# Fungsi untuk merekomendasikan makanan menggunakan KNN
def recommend_meal(nutrient_needs, k=3):
    # Normalisasi data makanan dan kebutuhan nutrisi
    food_nutrition = food_data[:, 1:].astype(float)
    normalized_food = (food_nutrition - np.mean(food_nutrition, axis=0)) / np.std(food_nutrition, axis=0)
    normalized_needs = (nutrient_needs - np.mean(food_nutrition, axis=0)) / np.std(food_nutrition, axis=0)
    
    # Inisialisasi dan fit KNN
    knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
    knn.fit(normalized_food)
    
    # Cari k tetangga terdekat
    distances, indices = knn.kneighbors([normalized_needs])
    
    # Pilih makanan secara acak dari k tetangga terdekat
    recommended_meal_index = random.choice(indices[0])
    
    return food_data[recommended_meal_index]

# Fungsi untuk membuat rekomendasi menu sehari
def generate_daily_menu(nutrient_needs):
    daily_menu = {
        "Sarapan": recommend_meal(nutrient_needs * np.array([0.25, 0.25, 0.25, 0.25])),
        "Snack Pagi": recommend_meal(nutrient_needs * np.array([0.1, 0.1, 0.1, 0.1])),
        "Makan Siang": recommend_meal(nutrient_needs * np.array([0.3, 0.3, 0.3, 0.3])),
        "Snack Sore": recommend_meal(nutrient_needs * np.array([0.1, 0.1, 0.1, 0.1])),
        "Makan Malam": recommend_meal(nutrient_needs * np.array([0.25, 0.25, 0.25, 0.25]))
    }
    return daily_menu

# Contoh penggunaan
age = 28
weight = 60  # kg
height = 165  # cm
trimester = 2
activity_level = "moderate"

nutrient_needs = calculate_nutrient_needs(age, weight, height, trimester, activity_level)
daily_menu = generate_daily_menu(nutrient_needs)

print(f"Kebutuhan gizi harian: {nutrient_needs}")
print("\nRekomendasi Menu Makanan:")
for meal, food in daily_menu.items():
    print(f"{meal}: {food[0]} ({food[1]} kal, {food[2]}g protein, {food[3]}g karbo, {food[4]}g lemak)")

Kebutuhan gizi harian: [2504.42, 125.221, 313.0525, 83.48066666666666]

Rekomendasi Menu Makanan:
Sarapan: Nasi Putih (200 kal, 4g protein, 40g karbo, 0g lemak)
Snack Pagi: Nasi Putih (200 kal, 4g protein, 40g karbo, 0g lemak)
Makan Siang: Nasi Putih (200 kal, 4g protein, 40g karbo, 0g lemak)
Snack Sore: Nasi Putih (200 kal, 4g protein, 40g karbo, 0g lemak)
Makan Malam: Yogurt (150 kal, 5g protein, 17g karbo, 8g lemak)


In [57]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import random

# Data dummy makanan
# Format: [nama, kalori, protein, karbohidrat, lemak, jenis_pangan, vitamin, mineral]
food_data = np.array([
    ["Nasi Putih", 200, 4, 40, 0, "makanan_pokok", 0, 0],
    ["Roti Gandum", 69, 3, 12, 1, "makanan_pokok", 2, 1],
    ["Kentang Rebus", 87, 2, 20, 0, "makanan_pokok", 1, 1],
    ["Ayam Panggang", 165, 31, 0, 3, "lauk_pauk", 0, 1],
    ["Ikan Salmon", 206, 22, 0, 13, "lauk_pauk", 2, 2],
    ["Tahu", 76, 8, 2, 5, "lauk_pauk", 1, 1],
    ["Telur Rebus", 70, 6, 0, 5, "lauk_pauk", 1, 1],
    ["Apel", 95, 0, 25, 0, "buah", 2, 1],
    ["Pisang", 105, 1, 27, 0, "buah", 2, 2],
    ["Jeruk", 62, 1, 15, 0, "buah", 3, 1],
    ["Sayur Bayam", 23, 3, 4, 0, "sayuran", 3, 3],
    ["Wortel", 52, 1, 12, 0, "sayuran", 4, 2],
    ["Brokoli", 31, 2, 6, 0, "sayuran", 3, 2],
    ["Susu", 150, 8, 12, 8, "minuman", 2, 3],
    ["Yogurt", 150, 5, 17, 8, "minuman", 1, 2],
    ["Kacang Tanah", 160, 7, 5, 14, "snack", 1, 2]
])

# Kalkulasi kebutuhan nutrisi harian
def calculate_nutrient_needs(age, weight, height, trimester, activity_level):
    bmr = 655 + (9.6 * weight) + (1.8 * height) - (4.7 * age)
    activity_factors = {"sedentary": 1.2, "light": 1.375, "moderate": 1.55, "active": 1.725}
    trimester_additions = {1: 0, 2: 340, 3: 450}
    total_calories = bmr * activity_factors[activity_level] + trimester_additions[trimester]
    carbs = (total_calories * 0.5) / 4
    protein = (total_calories * 0.2) / 4
    fat = (total_calories * 0.3) / 9
    return np.array([total_calories, protein, carbs, fat])

# Rekomendasi makanan menggunakan KNN
def recommend_food(nutrient_needs, food_type, portion=1, k=3):
    food_of_type = food_data[food_data[:, 5] == food_type]
    if len(food_of_type) == 0:
        return None
    
    food_nutrition = food_of_type[:, 1:5].astype(float)
    
    # Menangani nilai NaN
    food_nutrition = np.nan_to_num(food_nutrition, nan=0.0)
    nutrient_needs = np.nan_to_num(nutrient_needs, nan=0.0)
    
    # Hindari pembagian dengan nol saat normalisasi
    std_dev = np.std(food_nutrition, axis=0)
    std_dev[std_dev == 0] = 1  # Ganti nilai 0 dengan 1 untuk menghindari pembagian dengan nol
    
    normalized_food = (food_nutrition - np.mean(food_nutrition, axis=0)) / std_dev
    normalized_needs = (nutrient_needs - np.mean(food_nutrition, axis=0)) / std_dev
    
    knn = NearestNeighbors(n_neighbors=min(k, len(food_of_type)), metric='euclidean')
    knn.fit(normalized_food)
    
    distances, indices = knn.kneighbors([normalized_needs])
    recommended_food_index = random.choice(indices[0])
    
    recommended_food = food_of_type[recommended_food_index]
    recommended_food[1:5] = recommended_food[1:5].astype(float) * portion
    return recommended_food

# Membuat meal berdasarkan rekomendasi makanan
def generate_meal(nutrient_needs):
    meal = {
        "makanan_pokok": recommend_food(nutrient_needs * np.array([2/6, 0.25, 0.5, 0.25]), "makanan_pokok", 2/6),
        "lauk_pauk": recommend_food(nutrient_needs * np.array([1/6, 0.5, 0.25, 0.25]), "lauk_pauk", 1/6),
        "buah": recommend_food(nutrient_needs * np.array([1/6, 0.1, 0.4, 0.1]), "buah", 1/6),
        "sayuran": recommend_food(nutrient_needs * np.array([2/6, 0.15, 0.25, 0.1]), "sayuran", 2/6)
    }
    return meal

# Membuat menu harian
def generate_daily_menu(nutrient_needs):
    daily_menu = {
        "Sarapan": generate_meal(nutrient_needs * np.array([0.3, 0.3, 0.3, 0.3])),
        "Makan Siang": generate_meal(nutrient_needs * np.array([0.4, 0.4, 0.4, 0.4])),
        "Makan Malam": generate_meal(nutrient_needs * np.array([0.3, 0.3, 0.3, 0.3]))
    }
    
    # Hitung total nutrisi
    total_nutrition = np.sum([np.sum([food[1:5].astype(float) for food in meal.values() if food is not None], axis=0) 
                              for meal in daily_menu.values()], axis=0)
    
    # Jika kurang, tambahkan snack
    if np.any(total_nutrition < nutrient_needs):
        snack = recommend_food((nutrient_needs - total_nutrition) * np.array([1, 0.2, 0.5, 0.3]), "snack")
        if snack is not None:
            daily_menu["Snack"] = {"snack": snack}
    
    return daily_menu

# Contoh penggunaan
age = 28
weight = 60  # kg
height = 165  # cm
trimester = 2
activity_level = "moderate"

nutrient_needs = calculate_nutrient_needs(age, weight, height, trimester, activity_level)
daily_menu = generate_daily_menu(nutrient_needs)

print(f"Kebutuhan gizi harian: {nutrient_needs}")
print("\nRekomendasi Menu Makanan:")
for meal, foods in daily_menu.items():
    print(f"\n{meal}:")
    for food_type, food in foods.items():
        if food is not None:
            food_values = food[1:5].astype(float)  # Convert to float for formatting
            print(f"  {food_type.capitalize()}: {food[0]} ({food_values[0]:.0f} kal, {food_values[1]:.1f}g protein, {food_values[2]:.1f}g karbo, {food_values[3]:.1f}g lemak)")

# Hitung dan tampilkan total nutrisi
total_nutrition = np.sum([np.sum([food[1:5].astype(float) for food in meal.values() if food is not None], axis=0) 
                          for meal in daily_menu.values()], axis=0)
print(f"\nTotal nutrisi: {total_nutrition[0]:.0f} kal, {total_nutrition[1]:.1f}g protein, {total_nutrition[2]:.1f}g karbo, {total_nutrition[3]:.1f}g lemak")
print(f"Persentase dari kebutuhan: {(total_nutrition / nutrient_needs * 100).astype(int)}%")


Kebutuhan gizi harian: [2504.42        125.221       313.0525       83.48066667]

Rekomendasi Menu Makanan:

Sarapan:
  Makanan_pokok: Roti Gandum (23 kal, 1.0g protein, 4.0g karbo, 0.3g lemak)
  Lauk_pauk: Tahu (13 kal, 1.3g protein, 0.3g karbo, 0.8g lemak)
  Buah: Pisang (18 kal, 0.2g protein, 4.5g karbo, 0.0g lemak)
  Sayuran: Wortel (17 kal, 0.3g protein, 4.0g karbo, 0.0g lemak)

Makan Siang:
  Makanan_pokok: Roti Gandum (23 kal, 1.0g protein, 4.0g karbo, 0.3g lemak)
  Lauk_pauk: Ayam Panggang (28 kal, 5.2g protein, 0.0g karbo, 0.5g lemak)
  Buah: Apel (16 kal, 0.0g protein, 4.2g karbo, 0.0g lemak)
  Sayuran: Wortel (17 kal, 0.3g protein, 4.0g karbo, 0.0g lemak)

Makan Malam:
  Makanan_pokok: Nasi Putih (67 kal, 1.3g protein, 13.3g karbo, 0.0g lemak)
  Lauk_pauk: Ayam Panggang (28 kal, 5.2g protein, 0.0g karbo, 0.5g lemak)
  Buah: Pisang (18 kal, 0.2g protein, 4.5g karbo, 0.0g lemak)
  Sayuran: Brokoli (10 kal, 0.7g protein, 2.0g karbo, 0.0g lemak)

Snack:
  Snack: Kacang Tanah (16

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import random

# Membaca data dari resisData.csv
food_data = pd.read_csv('resisData.csv')

# Pastikan data memiliki nama kolom yang sesuai
food_data.columns = ['nama_bahan', 'jenis_pangan', 'energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram']

# Kalkulasi kebutuhan nutrisi harian untuk ibu hamil
def calculate_nutrient_needs(age, weight, height, trimester, activity_level):
    bmr = 655 + (9.6 * weight) + (1.8 * height) - (4.7 * age)
    activity_factors = {"sedentary": 1.2, "light": 1.375, "moderate": 1.55, "active": 1.725}
    trimester_additions = {1: 0, 2: 340, 3: 450}
    total_calories = bmr * activity_factors[activity_level] + trimester_additions[trimester]
    carbs = (total_calories * 0.5) / 4
    protein_gram = (total_calories * 0.2) / 4
    lemak_gram = (total_calories * 0.3) / 9
    return np.array([total_calories, protein_gram, carbs, lemak_gram])

# Rekomendasi makanan menggunakan KNN
def recommend_food(nutrient_needs, jenis_pangan, allergies, k=3):
    food_of_type = food_data[food_data['jenis_pangan'] == jenis_pangan]
    if food_of_type.empty:
        return None
    
    # Filter makanan berdasarkan alergi
    food_of_type = food_of_type[~food_of_type['nama_bahan'].isin(allergies)]
    
    if food_of_type.empty:
        return None
    
    food_nutrition = food_of_type[['energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram']].values.astype(float)
    
    food_nutrition = np.nan_to_num(food_nutrition, nan=0.0)
    nutrient_needs = np.nan_to_num(nutrient_needs, nan=0.0)
    
    std_dev = np.std(food_nutrition, axis=0)
    std_dev[std_dev == 0] = 1
    
    normalized_food = (food_nutrition - np.mean(food_nutrition, axis=0)) / std_dev
    normalized_needs = (nutrient_needs - np.mean(food_nutrition, axis=0)) / std_dev
    
    knn = NearestNeighbors(n_neighbors=min(k, len(food_of_type)), metric='euclidean')
    knn.fit(normalized_food)
    
    distances, indices = knn.kneighbors([normalized_needs])
    recommended_food_index = random.choice(indices[0])
    
    recommended_food = food_of_type.iloc[recommended_food_index]
    return recommended_food

# Membuat meal berdasarkan rekomendasi makanan
def generate_meal(nutrient_needs, meal_percentage, trimester, allergies):
    meal = {
        "makanan_pokok": recommend_food(nutrient_needs * meal_percentage * np.array([1, 0.25, 0.5, 0.25]), "makanan_pokok", allergies),
        "lauk_pauk": recommend_food(nutrient_needs * meal_percentage * np.array([1, 0.5, 0.25, 0.25]), "lauk_pauk", allergies),
        "buah": recommend_food(nutrient_needs * meal_percentage * np.array([1, 0.1, 0.4, 0.1]), "buah", allergies),
        "sayuran": recommend_food(nutrient_needs * meal_percentage * np.array([1, 0.15, 0.25, 0.1]), "sayuran", allergies)
    }
    
    if trimester == 3:
        meal["snack"] = recommend_food(nutrient_needs * 0.1, "snack", allergies)
    
    return meal

def generate_daily_menu(nutrient_needs, trimester, allergies):
    meal_percentages = {
        "Sarapan": 0.25,
        "Makan Siang": 0.3,
        "Makan Malam": 0.3,
        "Snack": 0.15
    }

    daily_menu = {}
    for meal, percentage in meal_percentages.items():
        if meal != "Snack":
            daily_menu[meal] = generate_meal(nutrient_needs, percentage, trimester, allergies)
        else:
            snack = recommend_food(nutrient_needs * percentage, "snack", allergies)
            if snack is not None:
                daily_menu[meal] = {"snack": snack}

    return daily_menu

def calculate_total_nutrition(daily_menu):
    total_nutrition = np.sum([np.sum([food[['energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram']].values.astype(float) 
                                       for food in meal.values() if food is not None], axis=0) 
                              for meal in daily_menu.values()], axis=0)
    return total_nutrition

# Fungsi utama
def get_food_recommendations(age, weight, height, trimester, activity_level, allergies):
    nutrient_needs = calculate_nutrient_needs(age, weight, height, trimester, activity_level)
    daily_menu = generate_daily_menu(nutrient_needs, trimester, allergies)

    print(f"Kebutuhan gizi harian: {nutrient_needs}")
    print("\nRekomendasi Menu Makanan:")
    for meal, foods in daily_menu.items():
        print(f"\n{meal}:")
        for jenis_pangan, food in foods.items():
            if food is not None:
                food_values = food[['energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram']].values.astype(float)
                print(f"  {jenis_pangan.capitalize()}: {food['nama_bahan']} ({food_values[0]:.0f} kal, {food_values[1]:.1f}g protein_gram, {food_values[2]:.1f}g lemak, {food_values[3]:.1f}g karbo)")

    total_nutrition = calculate_total_nutrition(daily_menu)
    print(f"\nTotal nutrisi: {total_nutrition[0]:.0f} kal, {total_nutrition[1]:.1f}g protein_gram, {total_nutrition[2]:.1f}g lemak, {total_nutrition[3]:.1f}g karbo")
    print(f"Persentase dari kebutuhan: {(total_nutrition / nutrient_needs * 100).astype(int)}%")

    print("\nKontribusi setiap waktu makan:")
    for meal, foods in daily_menu.items():
        meal_nutrition = np.sum([food[['energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram']].values.astype(float) for food in foods.values() if food is not None], axis=0)
        percentage = (meal_nutrition / total_nutrition) * 100
        print(f"{meal}: {percentage[0]:.1f}%")

# Contoh penggunaan
age = 28
weight = 60  # kg
height = 165  # cm
trimester = 2
activity_level = "moderate"
allergies = ["apel segar"]

get_food_recommendations(age, weight, height, trimester, activity_level, allergies)
